# notebook for choosing chromo indeces



First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp
import re
import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict
from morphct.mobility_kmc import snap_molecule_indices

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp



def vmd_index_slicer(filepath):
    with open(filepath, "r") as f:
        index_ids = f.readlines()
    numbers = []
    
    for line in index_ids:
        thing = re.findall("\/([0-9]+)(?=[^\/]*$)", line)
        try:
            numbers.append(int(thing[0]))
        except:
            pass
        
    indeces=[]
    
    for i in numbers:
        if i not in indeces:
            indeces.append(i)
            
    return(indeces)

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [22]:
gsd_file = "/Users/jimmy/cmelab/data/ITIC/887c520f567aa40dd310a8e1e0c0d6c4/trajectory.gsd"

with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[0]
    end_snap = f[-1]

    

ref_distance = 3.563594872561358

start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance

box = start_snap.configuration.box[:3]
#unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
#unwrap_snap = deepcopy(start_snap)
#unwrap_snap.particles.position = unwrapped_positions
#unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
#comp = from_snapshot(unwrap_snap, scale=0.1)
#comp.visualize().show()



In [3]:
print(start_snap.configuration.box)
print(end_snap.configuration.box)

[708.5166 708.5166 708.5166   0.       0.       0.    ]
[141.70389 141.70389 141.70389   0.        0.        0.     ]


In [20]:
indeces = vmd_index_slicer("/Users/jimmy/cmelab/data/ITIC/887c520f567aa40dd310a8e1e0c0d6c4/backbone_ids.txt")
gsd_mol_index = snap_molecule_indices(start_snap)
k = np.count_nonzero(gsd_mol_index==1)
for i in range(len(indeces)):
    indeces[i] = indeces[i]%k
chromo_ids = np.array(indeces)
print(chromo_ids)

[101 102 103  91  92 100  99  97  95  96  98  93  89  90  94  88  86  85
  87  59  60  58  57  55  56  52  53  51  54  25  26  24  23  21  18  22
  19  17  20  16  15   7   8   9   3   4   6  10  12  14  11  13   5   0
   1   2]


below takes the ids for random molecule and retures the first molecule ids

the guts for doin this live in the intramolecular notebook which i guess i didnt push???? come back to it latter

In [5]:
chromo2_ids = chromo_ids + k

In [6]:
aaids=[]

In [7]:
chromo2_ids = chromo2_ids + k

In [8]:
aaids.append(chromo_ids)
aaids.append(chromo2_ids)

In [9]:
print(aaids)

[array([101, 102, 103,  91,  92, 100,  99,  97,  95,  96,  98,  93,  89,
        90,  94,  88,  86,  85,  87,  59,  60,  58,  57,  55,  56,  52,
        53,  51,  54,  25,  26,  24,  23,  21,  18,  22,  19,  17,  20,
        16,  15,   7,   8,   9,   3,   4,   6,  10,  12,  14,  11,  13,
         5,   0,   1,   2]), array([309, 310, 311, 299, 300, 308, 307, 305, 303, 304, 306, 301, 297,
       298, 302, 296, 294, 293, 295, 267, 268, 266, 265, 263, 264, 260,
       261, 259, 262, 233, 234, 232, 231, 229, 226, 230, 227, 225, 228,
       224, 223, 215, 216, 217, 211, 212, 214, 218, 220, 222, 219, 221,
       213, 208, 209, 210])]


In [16]:
master_list = []
sublist = chromo_ids
for i in range(len(np.unique(gsd_mol_index))):         
    master_list.append(sublist)
    sublist = [x + k for x in sublist]

In [24]:
chromo_list = []
for i,master_list in enumerate(master_list):
    chromo_list.append(chromophores.Chromophore(i, start_snap, aaid, "acceptor", amber_dict))

In [23]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, end_snap, amber_dict, d_cut=min(box)/2)
print(f"There are {len(qcc_pairs)} chromophore pairs")

AttributeError: 'numpy.ndarray' object has no attribute 'center'

In [12]:
print(chromo_list)

[Chromophore 0 (acceptor): 56 atoms at 139.741 287.183 -123.293, Chromophore 1 (acceptor): 56 atoms at -336.041 -57.642 307.417]


In [15]:
qcc_input = eqcc.write_qcc_inp(end_snap, chromo2_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)

HOMO-1, HOMO, LUMO, LUMO+1
[-7.54844769 -7.37586757 -2.51095725 -2.1531804 ]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
i =  0# try any number from 0 to 180
print(f"Pair #{i}:")
visualize_qcc_input(qcc_pairs[i][1])

Pair #0:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")

In [ ]:
%%time
data = eqcc.singles_homolumo(chromo_list, "singles.txt")

In [ ]:
%%time
eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")

In [ ]:
print(chromo_list)

In [ ]:
i = 0
chromo = chromo_list[i]
print(f"Chromophore {i}:")
print(f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} LUMO+1: {chromo.lumo_1:.2f}")
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")